In [ ]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
from scipy.signal import butter, freqz, filtfilt, firwin, iirnotch, lfilter, find_peaks

In [ ]:
def pull_data(dir_name, file_name):
    f = open(dir_name + '/' + file_name + '.csv')
    xs = []
    ys = []
    zs = []
    rs = []
    timestamps = []
    for line in f:
        value = line.split(',')
        if len(value) > 3:
            timestamps.append(float(value[-4]))
            x = float(value[-3])
            y = float(value[-2])
            z = float(value[-1])
            r = math.sqrt(x ** 2 + y ** 2 + z ** 2)
            xs.append(x)
            ys.append(y)
            zs.append(z)
            rs.append(r)
    return np.array(xs), np.array(ys), np.array(zs), np.array(rs), np.array(timestamps)


In [ ]:
# read accelerometer data file

accel_file = 'data'
x_data, y_data, z_data, signal, timestamps = pull_data(accel_file, 'fall')

c = timestamps[0]
timestamps = (timestamps - c)/1000000000

In [ ]:
# FILL IN CODE: plot three-axis acceleration
plt.figure(figsize=(10,5))
pl.plot(timestamps, z_data, 'red', label = 'z-axis')
pl.plot(timestamps, x_data, 'blue', label = 'x-axis')
pl.plot(timestamps, y_data, 'green', label = 'y-axis')

plt.legend(loc='upper right')
pl.grid()
pl.show()

In [ ]:
# FILL IN CODE: Calculate and plot magnitude
magnitude = np.sqrt(x_data**2 + y_data**2 + z_data**2)
plt.figure(figsize=(10,5))
pl.plot(timestamps, magnitude, 'r-',label='magnitude')
plt.title("magnitude")
pl.grid()
pl.show()

In [ ]:
cutoff = 1
fs = 50.0  
order = 5
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq
b, a = butter(order, normal_cutoff, btype='low', analog=False)
BWfiltered = filtfilt(b, a, signal)

In [ ]:
plt.figure(figsize=(10,5))
pl.plot(timestamps, signal, 'r-',label='unfiltered')
pl.plot(timestamps, BWfiltered, 'b-', label='filtered')
plt.title("butterworth filter")
plt.legend(loc='upper left')
pl.grid()
pl.show()


plt.figure(figsize=(10,5))
pl.plot(timestamps, BWfiltered, 'b-', label='filtered')
plt.title("butterworth filter")
plt.legend(loc='upper right')
pl.grid()
pl.show()

In [ ]:

steps,_ = find_peaks(BWfiltered, height=8) 

print(f"{len(timestamps[steps])} steps")

In [ ]:
plt.figure(figsize=(10,5))
pl.plot(timestamps[steps], BWfiltered[steps], 'o')
pl.plot(timestamps, BWfiltered, 'b-', label='filtered')
plt.title("butterworth filter")
pl.grid()
pl.show()